# Load sample data

## From raw 84,000 dataset

In [1]:
from dask.distributed import Client, LocalCluster

In [2]:
dask_client = Client(LocalCluster(
    n_workers=20,
    threads_per_worker=1
))

2022-07-19 16:06:59,966 - distributed.diskutils - INFO - Found stale lock file and directory '/home/eeisenst/workspace/compassionai/garland/notebooks/dask-worker-space/worker-wrs5z3pq', purging


In [3]:
dask_client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 20
Total threads: 20,Total memory: 31.17 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:40311,Workers: 20
Dashboard: http://127.0.0.1:8787/status,Total threads: 20
Started: Just now,Total memory: 31.17 GiB
Comm: tcp://127.0.0.1:42707,Total threads: 1
Dashboard: http://127.0.0.1:37621/status,Memory: 1.56 GiB
Nanny: tcp://127.0.0.1:38207,


In [4]:
from cai_common.data import ParallelTMXLoader

In [5]:
parallel_df = ParallelTMXLoader() \
    .apply_markup() \
    .clean_bad_chars() \
    .dataframe

In [6]:
parallel_df = parallel_df[parallel_df.tohoku.str.isnumeric()]

In [7]:
parallel_df["tohoku"] = parallel_df.tohoku.fillna(-1).astype(int)

In [8]:
parallel_df = dask_client.persist(parallel_df)[["tohoku", "tibetan", "english"]]

In [9]:
sample_texts = parallel_df.english.compute().tolist()

## From processed data

In [2]:
import os

In [5]:
with open(os.path.join(os.environ['CAI_DATA_BASE_PATH'], "processed_datasets/84000-parallel-sentences-raw-no-dict/train.en")) as f:
    sample_texts = f.readlines()

In [7]:
sample_texts = [t.replace('\n', '') for t in sample_texts]

## Show some examples

In [8]:
sample_texts[:10]

['blessed one, this is how we train with our armor in the world.',
 'are both explained by the wise',
 'act like stairs and bridges.',
 'pausing on the seventh step,',
 'monks, virtuous acts must be engaged in with respect,',
 'he whose consciousness pursues signs is not interested in the dharma but is interested in signs.',
 'royal master of fragrances will be foremost in terms of miraculous abilities.',
 'moreover, ananda, you have revered me with benevolent actions of body, benevolent actions of speech, and benevolent actions of mind.',
 'monk, for these reasons, these phenomena are exactly as i have taught.',
 'when he had said this,']

# MNLI

## Load

In [9]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

In [10]:
# model_name = "bert-base-uncased"
model_name = "typeform/distilbert-base-uncased-mnli"

In [11]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
zsc = pipeline("zero-shot-classification", tokenizer=tokenizer, model=model)

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


In [12]:
model.config.label2id

{'ENTAILMENT': 0, 'NEUTRAL': 1, 'CONTRADICTION': 2}

## Sample with pipeline

In [ ]:
import random

In [ ]:
base_sent = random.choice(sample_texts)
candidate_sents = random.choices(sample_texts, k=10)
zsc(base_sent, candidate_sents, hypothesis_template="{}")

## Sample directly

In [13]:
import json
import random

from tqdm.auto import tqdm

import numpy as np

In [14]:
def run_scores(base_sent, candidate_sents, pipeline, hypothesis_template="{}"):
    model_outputs = []
    for inputs in pipeline.preprocess(base_sent, candidate_labels=candidate_sents, hypothesis_template=hypothesis_template):
        candidate_label = inputs["candidate_label"]
        sequence = inputs["sequence"]
        model_inputs = {k: inputs[k] for k in zsc.tokenizer.model_input_names}
        outputs = pipeline.model(**model_inputs)

        model_outputs.append({
            "candidate_label": candidate_label,
            "sequence": sequence,
            "is_last": inputs["is_last"],
            **outputs,
        })
    candidate_labels = [outputs["candidate_label"] for outputs in model_outputs]
    sequences = [outputs["sequence"] for outputs in model_outputs]
    logits = np.concatenate([output["logits"].detach().numpy() for output in model_outputs])
    N = logits.shape[0]
    n = len(candidate_labels)
    num_sequences = N // n
    reshaped_outputs = logits.reshape((num_sequences, n, -1))

    # softmax the "entailment" logits over all candidate labels
    contra_id = pipeline.model.config.label2id['CONTRADICTION']
    entail_logits = reshaped_outputs[..., pipeline.entailment_id]
    entail_scores = np.exp(entail_logits) / np.exp(entail_logits).sum(-1, keepdims=True)
    contra_logits = reshaped_outputs[..., contra_id]
    contra_scores = np.exp(contra_logits) / np.exp(contra_logits).sum(-1, keepdims=True)

    entail_top_inds = list(reversed(entail_scores[0].argsort()))
    contra_top_inds = list(reversed(contra_scores[0].argsort()))
    return {
        "sequence": sequences[0],
        "entailment": {
            "labels": [candidate_labels[i] for i in entail_top_inds],
            "scores": entail_scores[0, entail_top_inds].tolist()
        },
        "contradiction": {
            "labels": [candidate_labels[i] for i in contra_top_inds],
            "scores": contra_scores[0, contra_top_inds].tolist()
        }
    }

In [42]:
base_sent = random.choice(sample_texts)
candidate_sents = random.choices(sample_texts, k=10)
score_dict = run_scores(base_sent, candidate_sents, zsc)
print(json.dumps(score_dict, indent=4))

{
    "sequence": "cultivates their roots of virtue.",
    "entailment": {
        "labels": [
            "and in the same vein, , up to and including the eighty minor marks, are non-apprehensible. abiding in the emptiness of inherent existence, with regard to which this same refrain extensively applies,",
            "through this he settled into the second meditative concentration, which is free of thought and analysis and is imbued with the joy and pleasure born of meditative absorption.",
            "on this point the bhagavan",
            "evil is not produced and good is not destroyed.",
            "through the blessed one he realized the dharma.",
            "the relics of the jina are beautified by the clear sound of bells, the beauty of silken streamers, and bells shaken by the breeze.",
            "they will have sharp faculties in the jinas teachings",
            "the full extent of the temple-palace represents the buddhas abode.",
            "an act to defeat bodhis

In [43]:
sum(score_dict["entailment"]["scores"]), sum(score_dict["contradiction"]["scores"])

(1.000000037252903, 0.9999999590218067)

In [17]:
preprocessed = list(zsc.preprocess(base_sent, candidate_labels=candidate_sents, hypothesis_template="{}"))

In [18]:
model_outputs = []
for inputs in tqdm(preprocessed):
    candidate_label = inputs["candidate_label"]
    sequence = inputs["sequence"]
    model_inputs = {k: inputs[k] for k in zsc.tokenizer.model_input_names}
    outputs = zsc.model(**model_inputs)

    model_outputs.append({
        "candidate_label": candidate_label,
        "sequence": sequence,
        "is_last": inputs["is_last"],
        **outputs,
    })

  0%|          | 0/10 [00:00<?, ?it/s]

In [27]:
candidate_labels = [outputs["candidate_label"] for outputs in model_outputs]
sequences = [outputs["sequence"] for outputs in model_outputs]
logits = np.concatenate([output["logits"].detach().numpy() for output in model_outputs])
N = logits.shape[0]
n = len(candidate_labels)
num_sequences = N // n
reshaped_outputs = logits.reshape((num_sequences, n, -1))

# # softmax over the entailment vs. contradiction dim for each label independently
# entailment_id = zsc.entailment_id
# contradiction_id = -1 if entailment_id == 0 else 0
# entail_contr_logits = reshaped_outputs[..., [contradiction_id, entailment_id]]
# scores = np.exp(entail_contr_logits) / np.exp(entail_contr_logits).sum(-1, keepdims=True)
# scores = scores[..., 1]

# softmax the "entailment" logits over all candidate labels
entailment_id = 0#zsc.entailment_id
entail_logits = reshaped_outputs[..., entailment_id]
scores = np.exp(entail_logits) / np.exp(entail_logits).sum(-1, keepdims=True)

top_inds = list(reversed(scores[0].argsort()))
results = {
    "sequence": sequences[0],
    "labels": [candidate_labels[i] for i in top_inds],
    "scores": scores[0, top_inds].tolist(),
}

results

{'sequence': 'and their minds were unimpeded.',
 'labels': ['What is grasping?',
  'Because he is an expert in the entire commentarial literature, he has been able to restrain beings in their desire to argue and instead has promoted his own Dharma-precepts in a tactful way.',
  'there are also some sons of enlightened heritage or daughters of enlightened heritage who receive, retain, master, and recite this transcendent perfection of wisdom,',
  'This is the time when the Bliss-Gone One attains immortality,',
  'Then the Teacher would be the same As a deva or a human. {9}',
  '“Like the moon in the water,',
  'Unable to tolerate his condition, he ran to and fro.',
  'I should be equipoised in meditative concentration in this manner,',
  'When the Blessed One was in Rājagṛha, King Pradyota reigned in Ujjayinī,',
  'This son of noble family is someone whom all sentient beings, with the exception of the tathāgatas, should worship with every supreme offering.'],
 'scores': [0.8948529362678